Bahdanau et al, 2015 
Attention NN on machine translation

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split

import unicodedata
import re
import numpy as np
import os
import io
import time

# 数据集

FROM： http://www.manythings.org/anki/

数据预处理：



In [ ]:
path_to_zip = tf.keras.utils.get_file(
    'spa-eng.zip', origin='http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip',
    extract=True
)

path_to_file = os.path.dirname(path_to_zip) + "/spa-eng/spa.txt"

2646016/2638744 [==============================] - 0s 0us/step


In [ ]:
# 将 unicode 文件转化为 ascii
def unicode_to_ascii(s):
  return ''.join(c fro c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')

# 预处理：
#
def preprocess_sentence(w):
  w = unicode_to_ascii(w.lower().strip())
  # 1. strip 函数去除w前后的空格
  
  w = re.sub(r"([?.!,¿])", r" \1 ", w)
  # 2. 在单词与跟在其后的标点符号之间插入一个空格
  w = re.sub(r'[" "]+', " ", w)
  # 3. 重复的空格变为删除
  w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)
  # 4. 除了(a-z, A-Z, and 常用标点) 的其他字符替换为空格

  # strip() 删除开头和结尾的空格
  # rstrip 删除字符串右端的空格
  w = w.rstrip().strip() # 和w.strip()有什么区别。
  # 5. 删除多余空格

  w = '<start> ' + w + '<end>'
  # 6.  添加开始和结束标记
  return w


# 函数声明中使用的单个星号允许从调用环境传递可变数量的参数。 
# 在函数内部，它的行为就像一个元组。
# eg. zip(*wordpairs) 允许可变数量的参数
def create_dataset(path, num_examples):
  lines = io.open(path, encoding='UTF-8').read().strip().split('\n')

  word_pairs = [[preprocess_sentence(w) for w in l.split('\t')] 
                for l in lines[:num_examples]]

  return zip(*word_pairs)

In [ ]:
en, sp = create_dataset(path_to_file, None)
print(en[-1])
print(sp[-1])

In [ ]:
def max_length(tensor):
  # get maxlength for padding
  return max(len(t) for t in tensor)


def tokenize(lang):
  lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
  lang_tokenizer.fit_on_text(lang)

  tensor = lang_tokenizer.texts_to_sequences(lang)
  tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')

  return tensor, lang_tokenizer

def load_dataset(path, num_examples=None):
  # 输入输出对
  targ_lang, inp_lang = create_dataset(path, num_examples)

  input_tensor, inp_lang_tokenizer = tokenize(inp_lang)
  target_tensor, targ_lang_tokenizer = tokenize(targ_lang)

  return input_tensor, target_tensor, inp_lang_tokenizer, targ_lang_tokenizer

In [ ]:
# 创建一个tf.data 数据集
